In [2]:
import numpy as np
import cv2 as cv
import time

In [12]:
cap = cv.VideoCapture(1)

time.sleep(3) #Give some time to the camera to warmup
background = 0

#Capture the background. 40 frames so that when you have to superimpose the background on the cloak, it will
#take the average of those 40 frames for better accuracy and noise removal.
for i in range(40):
    ret, background = cap.read()

#For capturing the foreground. If you use while True here, the code will keep running even when the window is closed.
while (cap.isOpened()):
    ret, img = cap.read()#ret is a boolean value.It will be true if camera is capturing image and false otherwise. img is what the camera sees
                         #cap.read returns 2 values - image and the return value(true/false)
    if ret==False:
        break
    
    hsvimg = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    
    #Give the range depending on the colour of the cloak
    lower_col = np.array([0,120, 70]) #in opencv, we have reduced the colour angle by half. so instead of 0-360, it is now 0-180
    upper_col = np.array([0, 255, 255])
    mask1 = cv.inRange(hsvimg, lower_col, upper_col) #separating the cloak part
    
    lower_rng = np.array([170,120, 70])
    upper_rng = np.array([180, 255, 255])
    
#     164 132 47
#     172 241 239
    mask2 = cv.inRange(hsvimg, lower_rng, upper_rng)
    
    mask1 = mask1 + mask2 #add the two masks. This will comprise of the cloak area
    
    #Noise removal
    mask1 = cv.morphologyEx(mask1, cv.MORPH_OPEN, np.ones((3,3),np.uint8), iterations=2)#image, function to be performed, kernal/filter (remember cnn filter?)
    mask1 = cv.morphologyEx(mask1, cv.MORPH_DILATE, np.ones((3,3), np.uint8), iterations=1)
    
    mask2 = cv.bitwise_not(mask1) #This comprises of the remaining area. Everything except the cloak
    
    res1 = cv.bitwise_and(background, background, mask=mask1)#segmentation of colour
    res2 = cv.bitwise_and(img, img, mask=mask2)#substituting the cloak part
    final_output = cv.addWeighted(res1, 1, res2, 1, 0) #blending the two images
    
    cv.imshow("Your very own invisibility cloak!!", final_output)
    if cv.waitKey(10) == 27: #27 is esc key
        break

cap.release()
cv.destroyAllWindows()